### Editorial daily Summary - Getting data from ET API

In [107]:
## Libraries

from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import datetime

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import time
import lxml.etree as et

In [108]:
# Beautiful Soup
res = requests.get("https://economictimes.indiatimes.com/newsfeed_datewise.cms?date=4&month=3&year=2020")
soup = BeautifulSoup(res.content,'lxml')

In [109]:
# Agency Data
table1 = soup.find_all('agency')
cc="agency"
agency=[]
for x in table1:
    if cc in str(x):
        agency.append(str(x).split("agency>")[1].split("</")[0])

In [110]:
# MSID Data
table2 = soup.find_all('newsitemid')
cc="newsitemid"
msid=[]
for x in table2:
    if cc in str(x):
        msid.append(str(x).split("newsitemid>")[1].split("</")[0])

In [111]:
# Section Data
table3 = soup.find_all('weburl')
cc="weburl"
section=[]
for x in table3:
    if cc in str(x):
        section.append(str(x).split("weburl>")[1].split("/")[3])

In [112]:
# Comments Data: Step1

table4 = soup.find_all('commentcounturl')
cc="commentcounturl"
comment=[]
for x in table4:
    if cc in str(x):
        comment.append(str(x).split("commentcounturl")[1].split(">")[1].split("<")[0])

In [113]:
# Comments Data: Step 2

com2=[]
for w in comment:
    page=requests.get(w)
    soup=BeautifulSoup(page.content,'html.parser')
    artist_name_list_items = soup.find_all('CommentCount')
    y="CommentCount"
    for y in soup:
        com2.append(str(y).split(":")[2].split("\n")[0].replace('"',''))

In [114]:
# Creating DF with all attributes

df=pd.DataFrame(msid)
df.rename(columns={0:"msid"},inplace=True)
df["agency"]=agency
df["section"]=section
df["comments"]=com2

In [115]:
df['comments']=df['comments'].replace('', 0)
df.comments=df.comments.astype(int)
#.fillna(0,inplace=True)

-------------------------------------------------------------------------------------------------------------------------------------

#### Getting Data from GA

In [51]:
# Do the following fro Web, Wap and App

# 1) Direct
    #Landing Page + Source/Medium
    #Filter: Landing Page: articleshow
    #Filter:Page: articleshow

# 2) From HP
    #Filter:Page: articleshow
    #previous page path =home
    #Pagepath
    
    #2a)#Filter:Page: articleshow
        #previous page path =home
        #Filter Event Category and Event Action (Top News)
        #Pagepath
    #2b)#Filter:Page: articleshow
        #previous page path =home
        #Filter Event Category and Event Action (Latest News)
        #Pagepath
    #2c)#Filter:Page: articleshow
        #previous page path =home
        #Filter Event Category and Event Action (Recommended)
        #Pagepath
    
# 3) From other articles
    #Filter:Page: articleshow
    #previous page path =articleshow
    #Pagepath

#### Desktop Avg Time on Page

In [117]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:avgTimeOnPage'}],
                  'dimensions': [{'name': 'ga:pagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "desktop"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "2020"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "Mar"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "04"
                                    }
                                ]
                            }
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:avgTimeOnPage",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [118]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_web_atp = pd.DataFrame(print_response(response))

df_web_atp.rename(columns={"ga:pagePath":"url",
                  "ga:avgTimeOnPage":"atp_web"},inplace=True)

      ga:avgTimeOnPage                                        ga:pagePath
0          2163.000000  economictimes.indiatimes.com/markets/stocks/ne...
1          1873.666667  economictimes.indiatimes.com/news/economy/poli...
2          1573.100000  m.economictimes.com/news/economy/policy/govern...
3          1530.500000  etdev8243.indiatimes.com/news/economy/indicato...
4          1241.650000  economictimes.indiatimes.com/news/politics-and...
5          1190.238095  economictimes.indiatimes.com/industry/banking/...
6          1074.390244  economictimes.indiatimes.com/news/economy/poli...
7           994.285714  economictimes.indiatimes.com/markets/stocks/ne...
8           948.700000  economictimes.indiatimes.com/industry/transpor...
9           948.700000  economictimes.indiatimes.com/news/economy/fina...
10          906.460784  economictimes.indiatimes.com/small-biz/startup...
11          895.104364  economictimes.indiatimes.com/news/economy/fina...
12          887.691358  economictimes.

In [119]:
df_web_atp["x1"]=df_web_atp['url'].str.find(".cms")-8
df_web_atp["x2"]=df_web_atp['url'].str.find(".cms")

df_web_atp['msid']=[x[y:z]for x,y,z in zip(df_web_atp.url,df_web_atp.x1,df_web_atp.x2)]

df2_web_atp=df_web_atp[df_web_atp.msid.apply(lambda x: x.isnumeric())]
df3_web_atp=pd.DataFrame(df2_web_atp.groupby(["msid"])["atp_web"].sum()).reset_index()

df_web_direct_finx=df.merge(df3_web_atp,on='msid',how='left').fillna(0)

In [121]:
df_web_direct_finx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 392
Data columns (total 5 columns):
msid        393 non-null object
agency      393 non-null object
section     393 non-null object
comments    393 non-null int32
atp_web     393 non-null float64
dtypes: float64(1), int32(1), object(3)
memory usage: 16.9+ KB


#### mWeb Avg Time on Page

In [122]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:avgTimeOnPage'}],
                  'dimensions': [{'name': 'ga:pagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "mobile"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:avgTimeOnPage",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [123]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_wap_atp = pd.DataFrame(print_response(response))

df_wap_atp.rename(columns={"ga:pagePath":"url",
                  "ga:avgTimeOnPage":"atp_wap"},inplace=True)

       ga:avgTimeOnPage                                        ga:pagePath
0           3216.100000  m.economictimes.com/news/defence/80-kg-high-gr...
1           2127.500000  m.economictimes.com/magazines/panache/14-foods...
2           2020.243902  m.economictimes.com/markets/market-moguls/coro...
3           1969.400000  m.economictimes.com/industry/services/retail/d...
4           1937.950000  m.economictimes.com/markets/stocks/news/cabine...
5           1851.800000  m.economictimes.com/wealth/invest/what-is-smal...
6           1828.500000  economictimes.indiatimes.com/tech/ites/cogniza...
7           1820.400000  m.economictimes.com/mf/analysis/best-large-cap...
8           1818.350000  m.economictimes.com/industry/banking/finance/b...
9           1815.300000  m.economictimes.com/industry/services/property...
10          1814.300000  m.economictimes.com/news/politics-and-nation/b...
11          1811.250000  m.economictimes.com/markets/stocks/news/q3-gdp...
12          1810.952381  

In [124]:
df_wap_atp["x1"]=df_wap_atp['url'].str.find(".cms")-8
df_wap_atp["x2"]=df_wap_atp['url'].str.find(".cms")

df_wap_atp['msid']=[x[y:z]for x,y,z in zip(df_wap_atp.url,df_wap_atp.x1,df_wap_atp.x2)]

df2_wap_atp=df_wap_atp[df_wap_atp.msid.apply(lambda x: x.isnumeric())]
df3_wap_atp=pd.DataFrame(df2_wap_atp.groupby(["msid"])["atp_wap"].sum()).reset_index()

df_web_direct_finx2=df_web_direct_finx.merge(df3_wap_atp,on='msid',how='left').fillna(0)

In [127]:
df_web_direct_finx2

,msid,agency,section,comments,atp_web,atp_wap
0,74483280,ET Bureau,news,20,561.339890,313.426080
1,74483279,ET CONTRIBUTORS,news,0,60.567901,1012.252381
2,74483229,AFP,news,0,841.091838,84.150000
3,74483171,ET Bureau,news,1,739.912385,359.996032
4,74483135,ET Bureau,news,0,178.785002,528.902469
5,74483162,ET Bureau,news,1,685.432091,515.272405
6,74483141,ET CONTRIBUTORS,news,0,363.883333,467.242857
7,74483022,PTI,nri,1,1656.747251,709.298334
8,74483005,ET Bureau,news,0,488.246957,74.114754
9,74479143,ET Bureau,industry,1,175.081504,926.419672


#### Desktop - Direct Traffic

In [128]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:Pageviews'},{'expression': 'ga:uniquePageviews'}],
                  'dimensions': [{'name': 'ga:landingPagePath'},{'name': 'ga:sourceMedium'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:landingPagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "desktop"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "2020"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "Mar"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "04"
                                    }
                                ]
                            }
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:Pageviews",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [129]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_web_dir = pd.DataFrame(print_response(response))

df_web_dir.rename(columns={"ga:Pageviews":"pv_web_dir",
                  "ga:Sessions":"sessions_web_dir",
                  "ga:Users":"users_web_dir",
                  "ga:landingPagePath":"url",
                  "ga:sourceMedium":"source/medium",
                  "ga:uniquePageviews":"upv_web_dir"},inplace=True)

      ga:Pageviews  ga:Sessions  ga:Users  \
0         299087.0      78552.0   75654.0   
1         170910.0      58605.0   56943.0   
2         104946.0      35009.0   32617.0   
3          61605.0          0.0   23535.0   
4          32921.0       8169.0    8109.0   
5          28705.0       8980.0    8778.0   
6          27792.0       7825.0    7480.0   
7          24164.0          0.0   10683.0   
8          22866.0          0.0    9994.0   
9          22785.0          0.0    9406.0   
10         20535.0       4662.0    4541.0   
11         20251.0       6913.0    6811.0   
12         18163.0       5818.0    5676.0   
13         18062.0       5088.0    5027.0   
14         16075.0          0.0    7480.0   
15         15123.0       3791.0    3669.0   
16         14879.0       4095.0    2574.0   
17         14494.0       5088.0    4926.0   
18         12872.0       5433.0    5372.0   
19         12812.0       2980.0    1824.0   
20         12690.0       4156.0    4034.0   
21        

In [130]:
df_web_dir["x1"]=df_web_dir['url'].str.find(".cms")-8
df_web_dir["x2"]=df_web_dir['url'].str.find(".cms")

df_web_dir['msid']=[x[y:z]for x,y,z in zip(df_web_dir.url,df_web_dir.x1,df_web_dir.x2)]

In [131]:
df_web_dir["source"]="Other"
df_web_dir.loc[df_web_dir["source/medium"].str.contains('ET',regex=True,na=False),"source"]="internal"
df_web_dir.loc[(df_web_dir["source/medium"].str.contains('referral',regex=True,na=False)),"source"]="referral"
df_web_dir.loc[(df_web_dir["source/medium"].str.contains('social',regex=True,na=False)),"source"]="social"
df_web_dir.loc[(df_web_dir["source/medium"].str.contains('organic',regex=True,na=False))|
               (df_web_dir["source/medium"].str.contains('google',regex=True,na=False)),"source"]="organic"
df_web_dir.loc[df_web_dir["source/medium"].str.contains('(direct)',regex=True,na=False),"source"]="direct"
df_web_dir.loc[df_web_dir["source/medium"].str.contains('notification',regex=True,na=False),"source"]="notification"

df_web_dir.loc[(df_web_dir["source/medium"].str.contains('newsletter',regex=True,na=False))|(df_web_dir["source/medium"].str.contains('email',regex=True,na=False)),"source"]="newsletter"
#foo[foo.b.str.contains('oo', regex= True, na=False)]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


In [132]:
df2_web_dir=df_web_dir[df_web_dir.msid.apply(lambda x: x.isnumeric())]
df3_web_dir=pd.DataFrame(df2_web_dir.groupby(["msid","source"])["users_web_dir"].sum()).reset_index()

In [133]:
df4_web_dir=df3_web_dir.pivot(index='msid',columns='source',values='users_web_dir').fillna(0).sort_values("organic",ascending=False).reset_index()
#df.pivot(index='foo', columns='bar', values=['baz', 'zoo'])

In [134]:
df4_web_dir.rename(columns={"Other":"users_web_other",
                            "direct":"users_web_direct",
                            "internal":"users_web_internal",
                            "newsletter":"users_web_newsletter",
                            "notification":"users_web_notification",
                            "organic":"users_web_organic",
                            "social":"users_web_social",
                            "referral":"users_web_referral"},inplace=True
                  )

In [135]:
df_web_direct_fin=df_web_direct_finx2.merge(df4_web_dir,on='msid',how='left').fillna(0)

#### mWeb Direct Traffic

In [138]:
# mWeb Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:Pageviews'},{'expression': 'ga:uniquePageviews'}],
                  'dimensions': [{'name': 'ga:landingPagePath'},{'name': 'ga:sourceMedium'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:landingPagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "mobile"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:Pageviews",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [139]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_wap_dir = pd.DataFrame(print_response(response))

df_wap_dir.rename(columns={"ga:Pageviews":"pv_wap_dir",
                  "ga:Sessions":"sessions_wap_dir",
                  "ga:Users":"users_wap_dir",
                  "ga:landingPagePath":"url",
                  "ga:sourceMedium":"source/medium",
                  "ga:uniquePageviews":"upv_wap_dir"},inplace=True)

       ga:Pageviews  ga:Sessions  ga:Users  \
0          682524.0     532008.0  449137.0   
1          479079.0     350861.0  301905.0   
2          348834.0     269531.0  243705.0   
3          261848.0     180559.0  162578.0   
4          233792.0     192722.0  183640.0   
5          210804.0     154449.0  143685.0   
6          200040.0     157754.0  149908.0   
7          136408.0      95216.0   87877.0   
8          130873.0     100790.0   97932.0   
9          112447.0      86256.0   73768.0   
10         111311.0      75127.0   72572.0   
11         107156.0      82769.0   80093.0   
12         106304.0      82647.0   76018.0   
13         105858.0      79384.0   75816.0   
14          90695.0      73890.0   70282.0   
15          88668.0      64443.0   58767.0   
16          84087.0      61524.0   60288.0   
17          84005.0      57511.0   51247.0   
18          79465.0      65477.0   60490.0   
19          78025.0      51855.0   47557.0   
20          69694.0      55362.0  

In [140]:
df_wap_dir["x1"]=df_wap_dir['url'].str.find(".cms")-8
df_wap_dir["x2"]=df_wap_dir['url'].str.find(".cms")

df_wap_dir['msid']=[x[y:z]for x,y,z in zip(df_wap_dir.url,df_wap_dir.x1,df_wap_dir.x2)]

In [141]:
df_wap_dir["source"]="Other"
df_wap_dir.loc[df_wap_dir["source/medium"].str.contains('ET',regex=True,na=False),"source"]="internal"
df_wap_dir.loc[(df_wap_dir["source/medium"].str.contains('referral',regex=True,na=False)),"source"]="referral"
df_wap_dir.loc[(df_wap_dir["source/medium"].str.contains('social',regex=True,na=False)),"source"]="social"
df_wap_dir.loc[(df_wap_dir["source/medium"].str.contains('organic',regex=True,na=False))|
               (df_wap_dir["source/medium"].str.contains('google',regex=True,na=False)),"source"]="organic"
df_wap_dir.loc[df_wap_dir["source/medium"].str.contains('(direct)',regex=True,na=False),"source"]="direct"
df_wap_dir.loc[df_wap_dir["source/medium"].str.contains('notification',regex=True,na=False),"source"]="notification"

df_wap_dir.loc[(df_wap_dir["source/medium"].str.contains('newsletter',regex=True,na=False))|(df_wap_dir["source/medium"].str.contains('email',regex=True,na=False)),"source"]="newsletter"
#foo[foo.b.str.contains('oo', regex= True, na=False)]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


In [142]:
df2_wap_dir=df_wap_dir[df_wap_dir.msid.apply(lambda x: x.isnumeric())]
df3_wap_dir=pd.DataFrame(df2_wap_dir.groupby(["msid","source"])["users_wap_dir"].sum()).reset_index()

df4_wap_dir=df3_wap_dir.pivot(index='msid',columns='source',values='users_wap_dir').fillna(0).sort_values("organic",ascending=False).reset_index()
#df.pivot(index='foo', columns='bar', values=['baz', 'zoo'])

In [143]:
df4_wap_dir.rename(columns={"Other":"users_wap_other",
                            "direct":"users_wap_direct",
                            "internal":"users_wap_internal",
                            "newsletter":"users_wap_newsletter",
                            "notification":"users_wap_notification",
                            "organic":"users_wap_organic",
                            "social":"users_wap_social",
                            "referral":"users_wap_referral"},inplace=True
                  )

In [144]:
df_wap_direct_fin1=df_web_direct_fin.merge(df4_wap_dir,on='msid',how='left').fillna(0)

#### App Metrics

In [145]:
# app Traffic

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '78596802' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "78596802",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:screenviews'},{'expression': 'ga:uniqueScreenviews'}],
                  'dimensions': [{'name': 'ga:screenName'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:screenName",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:screenviews",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [146]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_app_dir = pd.DataFrame(print_response(response))

       ga:Sessions  ga:Users  \
0            929.0   38378.0   
1            618.0   28098.0   
2            509.0   25826.0   
3             10.0   24820.0   
4              6.0   24095.0   
5              9.0   23526.0   
6             13.0   22987.0   
7            353.0   21778.0   
8              7.0   21483.0   
9            410.0   20546.0   
10             9.0   20243.0   
11           256.0   17738.0   
12             4.0   19061.0   
13           313.0   17935.0   
14             7.0   17979.0   
15             5.0   16438.0   
16           393.0   15965.0   
17           333.0   15597.0   
18           285.0   14802.0   
19            14.0   15639.0   
20             4.0   15993.0   
21           274.0   15077.0   
22             7.0   15151.0   
23           232.0   13925.0   
24           299.0   13033.0   
25             2.0   13675.0   
26           204.0   12685.0   
27           243.0   12305.0   
28             3.0   11878.0   
29           228.0   12174.0   
...     

In [147]:
df_app_dir.rename(columns={"ga:screenviews":"pv_app_dir",
                           "ga:Sessions":"sessions_app_dir",
                           "ga:Users":"users_app_dir",
                           "ga:screenName":"url",
                           "ga:uniqueScreenviews":"upv_app_dir"},inplace=True)

In [148]:
df_app_dir["x1"]=df_app_dir['url'].str.find(".cms")-8
df_app_dir["x2"]=df_app_dir['url'].str.find(".cms")

df_app_dir['msid']=[x[y:z]for x,y,z in zip(df_app_dir.url,df_app_dir.x1,df_app_dir.x2)]

In [149]:
df2_app_dir=df_app_dir[df_app_dir.msid.apply(lambda x: x.isnumeric())]
df3_app_dir=pd.DataFrame(df2_app_dir.groupby(["msid"])["users_app_dir"].sum()).reset_index()

In [150]:
df_direct_fin2=df_wap_direct_fin1.merge(df3_app_dir,on='msid',how='left').fillna(0)

#### Traffic from other article

##### Desktop -other article

In [151]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:Pageviews'},{'expression': 'ga:uniquePageviews'}],
                  'dimensions': [{'name': 'ga:pagePath'},{'name': 'ga:previousPagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:previousPagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "desktop"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "2020"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "Mar"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "04"
                                    }
                                ]
                            }
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:Pageviews",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [152]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_web_art = pd.DataFrame(print_response(response))

df_web_art.rename(columns={"ga:Pageviews":"pv_web_art",
                           "ga:Sessions":"sessions_web_art",
                           "ga:Users":"users_web_art",
                           "ga:pagePath":"url",
                           "ga:previousPagePath":"prev_url",
                           "ga:uniquePageviews":"upv_web_art"},inplace=True)

       ga:Pageviews  ga:Sessions  ga:Users  \
0          187593.0          0.0   20920.0   
1           95013.0          0.0   11028.0   
2           55808.0          0.0    8169.0   
3           38475.0          0.0    4703.0   
4           17413.0          0.0    1885.0   
5           14210.0          0.0     953.0   
6           14068.0          0.0    1378.0   
7           13602.0          0.0    1460.0   
8           13298.0          0.0    1358.0   
9           12447.0          0.0     689.0   
10          12183.0          0.0    1054.0   
11          11778.0          0.0    1176.0   
12           9325.0          0.0     932.0   
13           9203.0          0.0     912.0   
14           8332.0          0.0     223.0   
15           8169.0          0.0     365.0   
16           8149.0          0.0    1520.0   
17           8149.0          0.0    1054.0   
18           7784.0          0.0    2696.0   
19           7683.0          0.0    1034.0   
20           7399.0          0.0  

In [153]:
df_web_art["x1"]=df_web_art['url'].str.find(".cms")-8
df_web_art["x2"]=df_web_art['url'].str.find(".cms")

df_web_art["y1"]=df_web_art['prev_url'].str.find(".cms")-8
df_web_art["y2"]=df_web_art['prev_url'].str.find(".cms")

df_web_art['msid']=[x[y:z]for x,y,z in zip(df_web_art.url,df_web_art.x1,df_web_art.x2)]

df_web_art['prev_msid']=[x[y:z]for x,y,z in zip(df_web_art.prev_url,df_web_art.y1,df_web_art.y2)]

df2_web_art=df_web_art[df_web_art.msid.apply(lambda x: x.isnumeric())]

df3_web_art=df2_web_art[df2_web_art.prev_msid.apply(lambda x: x.isnumeric())]

df4_web_art=df3_web_art.loc[~(df3_web_art["msid"]==df3_web_art["prev_msid"])]

df5_web_art=pd.DataFrame(df4_web_art.groupby(["msid"])["users_web_art"].sum()).reset_index()

df_web_artdir_fin=df_direct_fin2.merge(df5_web_art,on='msid',how='left').fillna(0)

##### mWeb-Other Article

In [154]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:Pageviews'},{'expression': 'ga:uniquePageviews'}],
                  'dimensions': [{'name': 'ga:pagePath'},{'name': 'ga:previousPagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:previousPagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "mobile"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:Users",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [155]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_wap_art = pd.DataFrame(print_response(response))

df_wap_art.rename(columns={"ga:Pageviews":"pv_wap_art",
                           "ga:Sessions":"sessions_wap_art",
                           "ga:Users":"users_wap_art",
                           "ga:pagePath":"url",
                           "ga:previousPagePath":"prev_url",
                           "ga:uniquePageviews":"upv_wap_art"},inplace=True)

       ga:Pageviews  ga:Sessions  ga:Users  \
0           20312.0          0.0  149969.0   
1           10298.0          0.0   69187.0   
2            2129.0          0.0   58950.0   
3           42570.0          0.0   44435.0   
4            5007.0          0.0   43888.0   
5           40807.0          0.0   43138.0   
6            7055.0          0.0   42226.0   
7           38030.0          0.0   31867.0   
8            1845.0          0.0   27022.0   
9            1906.0          0.0   24326.0   
10           2696.0          0.0   22988.0   
11           2291.0          0.0   21630.0   
12           2858.0          0.0   19886.0   
13          15528.0          0.0   18934.0   
14           3325.0          0.0   18285.0   
15          19988.0          0.0   17798.0   
16           2716.0          0.0   17129.0   
17          13602.0          0.0   17048.0   
18           1297.0          0.0   16785.0   
19            649.0          0.0   15873.0   
20            953.0          0.0  

In [156]:
df_wap_art["x1"]=df_wap_art['url'].str.find(".cms")-8
df_wap_art["x2"]=df_wap_art['url'].str.find(".cms")

df_wap_art["y1"]=df_wap_art['prev_url'].str.find(".cms")-8
df_wap_art["y2"]=df_wap_art['prev_url'].str.find(".cms")

df_wap_art['msid']=[x[y:z]for x,y,z in zip(df_wap_art.url,df_wap_art.x1,df_wap_art.x2)]

df_wap_art['prev_msid']=[x[y:z]for x,y,z in zip(df_wap_art.prev_url,df_wap_art.y1,df_wap_art.y2)]

df2_wap_art=df_wap_art[df_wap_art.msid.apply(lambda x: x.isnumeric())]

df3_wap_art=df2_wap_art[df2_wap_art.prev_msid.apply(lambda x: x.isnumeric())]

df4_wap_art=df3_wap_art.loc[~(df3_wap_art["msid"]==df3_wap_art["prev_msid"])]

df5_wap_art=pd.DataFrame(df4_wap_art.groupby(["msid"])["users_wap_art"].sum()).reset_index()

df_web_artdir_fin3=df_web_artdir_fin.merge(df5_wap_art,on='msid',how='left').fillna(0)

##### App-Other Article

In [159]:
# mWeb Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '78596802' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "78596802",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:screenviews'},{'expression': 'ga:uniqueScreenviews'}],
                  'dimensions': [{'name': 'ga:screenName'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:screenName",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:landingScreenName",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:screenviews",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [160]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_app_art = pd.DataFrame(print_response(response))


      ga:Sessions  ga:Users  \
0           921.0    1464.0   
1           714.0    1411.0   
2           493.0    1215.0   
3           589.0    1210.0   
4           581.0    1228.0   
5           611.0     989.0   
6           443.0    1099.0   
7           825.0     787.0   
8           501.0     913.0   
9           463.0     958.0   
10          435.0     589.0   
11           25.0     563.0   
12          772.0    1130.0   
13          491.0     486.0   
14          405.0     674.0   
15          337.0     677.0   
16          727.0    1074.0   
17          402.0     634.0   
18          352.0     611.0   
19          340.0     604.0   
20          249.0     475.0   
21          611.0     883.0   
22          563.0     838.0   
23          357.0     586.0   
24          327.0     591.0   
25          347.0     531.0   
26          272.0     486.0   
27          264.0     463.0   
28          236.0     413.0   
29          304.0     458.0   
...           ...       ...   
3497    

In [161]:
df_app_art.rename(columns={"ga:screenviews":"pv_app_art",
                           "ga:Sessions":"sessions_app_art",
                           "ga:Users":"users_app_art",
                           "ga:screenName":"url",
                           "ga:uniqueScreenviews":"upv_app_art"},inplace=True)

In [162]:
df_app_art["x1"]=df_app_art['url'].str.find(".cms")-8
df_app_art["x2"]=df_app_art['url'].str.find(".cms")

df_app_art['msid']=[x[y:z]for x,y,z in zip(df_app_art.url,df_app_art.x1,df_app_art.x2)]

df2_app_art=df_app_art[df_app_art.msid.apply(lambda x: x.isnumeric())]
df3_app_art=pd.DataFrame(df2_app_art.groupby(["msid"])["users_app_art"].sum()).reset_index()

df_web_artdir_fin4=df_web_artdir_fin3.merge(df3_app_art,on='msid',how='left').fillna(0)

#### Desk Home Page Traffic

In [163]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'}],
                  'dimensions': [{'name': 'ga:pagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "desktop"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:previousPagePath",
                                    "operator": "EXACT",
                                    "expressions": "economictimes.indiatimes.com/"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "2020"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "Mar"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:dimension8",
                                    "operator": "REGEXP",
                                    "expressions": "04"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:Users",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [164]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_web_hp_tot = pd.DataFrame(print_response(response))

     ga:Users                                        ga:pagePath
0      7967.0  economictimes.indiatimes.com/industry/telecom/...
1      7460.0  economictimes.indiatimes.com/industry/banking/...
2      6000.0  economictimes.indiatimes.com/markets/ipos/fpos...
3      4845.0  economictimes.indiatimes.com/markets/stocks/ne...
4      4581.0  economictimes.indiatimes.com/news/politics-and...
5      4521.0  economictimes.indiatimes.com/news/internationa...
6      4399.0  economictimes.indiatimes.com/industry/telecom/...
7      2615.0  economictimes.indiatimes.com/news/economy/poli...
8      2433.0  economictimes.indiatimes.com/industry/transpor...
9      2392.0  economictimes.indiatimes.com/markets/stocks/ne...
10     2210.0  economictimes.indiatimes.com/markets/stocks/ne...
11     2088.0  economictimes.indiatimes.com/news/economy/poli...
12     1946.0  economictimes.indiatimes.com/news/defence/us-h...
13     1906.0  economictimes.indiatimes.com/wealth/tax/you-ca...
14     1865.0  economicti

In [165]:
df_web_hp_tot.rename(columns={"ga:pagePath":"url",
                              "ga:Users":"users_web_hp_tot"},inplace=True)

df_web_hp_tot["x1"]=df_web_hp_tot['url'].str.find(".cms")-8
df_web_hp_tot["x2"]=df_web_hp_tot['url'].str.find(".cms")

df_web_hp_tot['msid']=[x[y:z]for x,y,z in zip(df_web_hp_tot.url,df_web_hp_tot.x1,df_web_hp_tot.x2)]

df2_web_hp_tot=df_web_hp_tot[df_web_hp_tot.msid.apply(lambda x: x.isnumeric())]
df3_web_hp_tot=pd.DataFrame(df2_web_hp_tot.groupby(["msid"])["users_web_hp_tot"].sum()).reset_index()

df_web_direct_fin5=df_web_artdir_fin4.merge(df3_web_hp_tot,on='msid',how='left').fillna(0)

#### Desk HP Traffic - Top News Widget

In [166]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:uniqueEvents'}],
                  'dimensions': [{'name': 'ga:eventLabel'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:eventAction",
                                    "operator": "REGEXP",
                                    "expressions": "Widget Top News"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:eventCategory",
                                    "operator": "REGEXP",
                                    "expressions": "HP Clicks"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "desktop"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:uniqueEvents",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [167]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_web_hp_top = pd.DataFrame(print_response(response))

                                          ga:eventLabel  ga:uniqueEvents
0                                            Tab Latest          36793.0
1                                               Tab Top          15731.0
2                                       Tab Recommended           3203.0
3     1 - https://economictimes.indiatimes.com/news/...            872.0
4     1 - https://economictimes.indiatimes.com/indus...            689.0
5     1 - https://economictimes.indiatimes.com/news/...            446.0
6     1 - /news/politics-and-nation/28-confirmed-cas...            304.0
7     21 - https://economictimes.indiatimes.com/maga...            284.0
8     2 - https://economictimes.indiatimes.com/indus...            264.0
9     1 - https://economictimes.indiatimes.com/indus...            243.0
10    1 - https://economictimes.indiatimes.com/news/...            243.0
11    More - https://economictimes.indiatimes.com/he...            243.0
12    1 - /industry/banking/finance/banking/yes-ban

In [168]:
df_web_hp_top.rename(columns={"ga:eventLabel":"url",
                              "ga:uniqueEvents":"uev_web_hp_top"},inplace=True)

df_web_hp_top["x1"]=df_web_hp_top['url'].str.find(".cms")-8
df_web_hp_top["x2"]=df_web_hp_top['url'].str.find(".cms")

df_web_hp_top['msid']=[x[y:z]for x,y,z in zip(df_web_hp_top.url,df_web_hp_top.x1,df_web_hp_top.x2)]

df2_web_hp_top=df_web_hp_top[df_web_hp_top.msid.apply(lambda x: x.isnumeric())]
df3_web_hp_top=pd.DataFrame(df2_web_hp_top.groupby(["msid"])["uev_web_hp_top"].sum()).reset_index()

df_web_direct_fin6=df_web_direct_fin5.merge(df3_web_hp_top,on='msid',how='left').fillna(0)

#### Desk HP Traffic - Latest News Widget

In [169]:
# Desktop Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:uniqueEvents'}],
                  'dimensions': [{'name': 'ga:eventLabel'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:eventAction",
                                    "operator": "REGEXP",
                                    "expressions": "Widget Latest News"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:eventCategory",
                                    "operator": "REGEXP",
                                    "expressions": "HP Clicks"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "REGEXP",
                                    "expressions": "desktop"
                                    }
                                ]
                            }
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:uniqueEvents",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [170]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_web_hp_lat = pd.DataFrame(print_response(response))

                                         ga:eventLabel  ga:uniqueEvents
0         More - https://economictimes.indiatimes.com/            284.0
1    1 - /industry/banking/finance/banking/yes-bank...            101.0
2    1 - /magazines/panache/corona-paranoia-zomato-...             81.0
3    1 - /mf/analysis/they-had-rs-50-lakh-in-fd-but...             81.0
4    1 - /magazines/panache/pizza-cough-green-phleg...             61.0
5    1 - /magazines/panache/whatsapp-launches-dark-...             61.0
6    1 - /markets/stocks/news/jp-morgan-cuts-target...             61.0
7    1 - /industry/auto/auto-news/anurag-mehrotra-t...             41.0
8    1 - /industry/banking/finance/banking/rbi-anno...             41.0
9    1 - /industry/media/entertainment/covid-19-pai...             41.0
10   1 - /industry/telecom/telecom-policy/fix-tarif...             41.0
11   1 - /industry/transportation/airlines-/-aviati...             41.0
12   1 - /markets/expert-view/sbi-card-valuations-e...          

In [171]:
df_web_hp_lat.rename(columns={"ga:eventLabel":"url",
                              "ga:uniqueEvents":"uev_web_hp_lat"},inplace=True)

df_web_hp_lat["x1"]=df_web_hp_lat['url'].str.find(".cms")-8
df_web_hp_lat["x2"]=df_web_hp_lat['url'].str.find(".cms")

df_web_hp_lat['msid']=[x[y:z]for x,y,z in zip(df_web_hp_lat.url,df_web_hp_lat.x1,df_web_hp_lat.x2)]

df2_web_hp_lat=df_web_hp_lat[df_web_hp_lat.msid.apply(lambda x: x.isnumeric())]
df3_web_hp_lat=pd.DataFrame(df2_web_hp_lat.groupby(["msid"])["uev_web_hp_lat"].sum()).reset_index()

df_web_direct_fin7=df_web_direct_fin6.merge(df3_web_hp_lat,on='msid',how='left').fillna(0)

#### mWeb HP Traffic

In [172]:
# mWeb Traffic - Direct Landing

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

# Web-wap - 16182921
# App - 78596802

'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2020-03-04', 'endDate': '2020-03-06'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Pageviews'}],
                  'dimensions': [{'name': 'ga:pagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": "articleshow"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:previousPagePath",
                                    "operator": "EXACT",
                                    "expressions": "m.economictimes.com/"
                                    }
                                ]
                            },
                        
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:deviceCategory",
                                    "operator": "EXACT",
                                    "expressions": "mobile"
                                    }
                                ]
                            }
                        
                        
                        
                        
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:Users",
                            "sortOrder": "DESCENDING"
                            }
                    ]
                    ,
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [173]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df_wap_hp = pd.DataFrame(print_response(response))

      ga:Pageviews  ga:Users  \
0          14656.0   12609.0   
1          12751.0   12426.0   
2           7278.0    6852.0   
3           5615.0    5575.0   
4           5250.0    5088.0   
5           4602.0    4480.0   
6           4602.0    4298.0   
7           4237.0    4156.0   
8           4237.0    3973.0   
9           3669.0    3385.0   
10          3243.0    3223.0   
11          3223.0    3122.0   
12          2899.0    2838.0   
13          2858.0    2818.0   
14          2919.0    2737.0   
15          2939.0    2737.0   
16          2818.0    2737.0   
17          2635.0    2615.0   
18          2473.0    2595.0   
19          2574.0    2534.0   
20          2473.0    2453.0   
21          2433.0    2352.0   
22          2331.0    2291.0   
23          2412.0    2270.0   
24          2230.0    2169.0   
25          2189.0    2169.0   
26          2189.0    2149.0   
27          2129.0    2088.0   
28          2047.0    2047.0   
29          2189.0    2027.0   
...     

In [177]:
df_wap_hp.rename(columns={"ga:pagePath":"url",
                          "ga:Users":"users_wap_hp_tot"},inplace=True)

df_wap_hp["x1"]=df_wap_hp['url'].str.find(".cms")-8
df_wap_hp["x2"]=df_wap_hp['url'].str.find(".cms")

df_wap_hp['msid']=[x[y:z]for x,y,z in zip(df_wap_hp.url,df_wap_hp.x1,df_wap_hp.x2)]

df2_wap_hp=df_wap_hp[df_wap_hp.msid.apply(lambda x: x.isnumeric())]
df3_wap_hp=pd.DataFrame(df2_wap_hp.groupby(["msid"])["users_wap_hp_tot"].sum()).reset_index()
df_web_direct_fin8=df_web_direct_fin7.merge(df3_wap_hp,on='msid',how='left').fillna(0)

In [178]:
df_web_direct_fin8.to_csv("01 04Mar2020 Daily Summary.csv",index=False)